<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/PRJCA002413_CRX118014_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRJCA002413_CRX118014

This notebook uses the result filtered count matrix from the following google colab notebook.
https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/PRJCA002413_CRX118014.ipynb Please run it and upload the file in /content

In [ ]:
# define the values for the analysis

# accession id for the data
id = "PRJCA002413"
samp_id = ["CRX118014"]

# If only bam available files, set bam = True, Fill link and filename
bam = False

# If fastq links available but are not ffq links
fastq_ffqlinks = False


if bam:
  # Assign link to python variable
  link_to_bam = ""
  
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename=""


if not bam:

  # Copy and paste the links from the ACE2 scRNAseq datasets google  spreadsheet
  links_raw = "ftp://download.big.ac.cn/gsa3/CRA002497/CRR143977/CRR143977_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143977/CRR143977_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143978/CRR143978_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143978/CRR143978_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143979/CRR143979_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143979/CRR143979_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143980/CRR143980_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143980/CRR143980_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143981/CRR143981_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143981/CRR143981_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143982/CRR143982_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143982/CRR143982_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143983/CRR143983_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143983/CRR143983_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143984/CRR143984_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143984/CRR143984_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143985/CRR143985_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143985/CRR143985_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143986/CRR143986_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143986/CRR143986_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143987/CRR143987_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143987/CRR143987_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143988/CRR143988_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143988/CRR143988_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143989/CRR143989_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143989/CRR143989_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143990/CRR143990_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143990/CRR143990_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143991/CRR143991_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143991/CRR143991_r2.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143992/CRR143992_f1.fastq.gz	ftp://download.big.ac.cn/gsa3/CRA002497/CRR143992/CRR143992_r2.fastq.gz"

  # Convert it to a list where each link is an element
  
  fastqs = links_raw.split()

#sample specific metadata

no_samples = 1

condition = ["healthy control"] * no_samples

In [ ]:
no_samples = 1

fastqs_per_sample = [4, 8, 8] 

sample_id = samp_id

database_id = [id] * no_samples

tissue = ["blood"] * no_samples

cell_type = ["PBMCs"] * no_samples

species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Wen et al 2020"] * no_samples

figure = ["Fig: all"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]



# Imports and installs

In [ ]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 59.1MB 116kB/s 
     |████████████████████████████████| 133kB 41.0MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 10.3MB 26.0MB/s 
     |████████████████████████████████| 13.2MB 42.2MB/s 
     |████████████████████████████████| 112kB 56.7MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 1.2MB 39.9MB/s 


# Downloads: (bam (if bam) and index

In [ ]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  !wget -- continue ${BAM_LINK}




In [ ]:
if bam:
  # Convert to fastq
  !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $bam_filename ./fastqs\

  # Remove original bam file to save space
  !rm $bam_filename


In [ ]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


In [ ]:
# Remove fastqs that wont be analyzed to save space
if bam:
  for elem in remov_elem:
    !rm $elem

In [ ]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [ ]:
# wget fastqs from non ffq links in fastqs folder
if not bam and not fastq_ffqlinks:
  !mkdir fastqs
  %cd fastqs
  while len(os.listdir()) != len(fastqs):
    for link in fastqs:
      !wget $link --continue

  # update fastqs variable with name of files
  fastqs = sorted(os.listdir())

/content/fastqs
--2021-04-28 16:53:17--  ftp://download.big.ac.cn/gsa3/CRA002497/CRR143977/CRR143977_f1.fastq.gz
           => ‘CRR143977_f1.fastq.gz’
Resolving download.big.ac.cn (download.big.ac.cn)... 124.16.164.229
Connecting to download.big.ac.cn (download.big.ac.cn)|124.16.164.229|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /gsa3/CRA002497/CRR143977 ... done.
==> SIZE CRR143977_f1.fastq.gz ... 780495252
==> PASV ... done.    ==> RETR CRR143977_f1.fastq.gz ... done.
Length: 780495252 (744M) (unauthoritative)

CRR143977_f1.fastq. 100%[===================>] 744.34M  11.1MB/s    in 69s     

2021-04-28 16:54:30 (10.8 MB/s) - ‘CRR143977_f1.fastq.gz’ saved [780495252]

--2021-04-28 16:54:30--  ftp://download.big.ac.cn/gsa3/CRA002497/CRR143977/CRR143977_r2.fastq.gz
           => ‘CRR143977_r2.fastq.gz’
Resolving download.big.ac.cn (download.big.ac.cn)... 124.16.164.229
Connecting to download.big.ac.cn

In [ ]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and fastq_ffqlinks:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and not fastq_ffqlinks:
  %cd /content/fastqs

  # Download the corresponding Kallisto index to fastq folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

/content/fastqs
[2021-04-28 19:25:08,758]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
100% 2.23G/2.23G [01:53<00:00, 21.1MB/s]
[2021-04-28 19:27:04,022]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [ ]:
# Check to make sure the metadata is in the right order after sorting
print(fastqs)

['CRR143977_f1.fastq.gz', 'CRR143977_r2.fastq.gz', 'CRR143978_f1.fastq.gz', 'CRR143978_r2.fastq.gz', 'CRR143979_f1.fastq.gz', 'CRR143979_r2.fastq.gz', 'CRR143980_f1.fastq.gz', 'CRR143980_r2.fastq.gz', 'CRR143981_f1.fastq.gz', 'CRR143981_r2.fastq.gz', 'CRR143982_f1.fastq.gz', 'CRR143982_r2.fastq.gz', 'CRR143983_f1.fastq.gz', 'CRR143983_r2.fastq.gz', 'CRR143984_f1.fastq.gz', 'CRR143984_r2.fastq.gz', 'CRR143985_f1.fastq.gz', 'CRR143985_r2.fastq.gz', 'CRR143986_f1.fastq.gz', 'CRR143986_r2.fastq.gz', 'CRR143987_f1.fastq.gz', 'CRR143987_r2.fastq.gz', 'CRR143988_f1.fastq.gz', 'CRR143988_r2.fastq.gz', 'CRR143989_f1.fastq.gz', 'CRR143989_r2.fastq.gz', 'CRR143990_f1.fastq.gz', 'CRR143990_r2.fastq.gz', 'CRR143991_f1.fastq.gz', 'CRR143991_r2.fastq.gz', 'CRR143992_f1.fastq.gz', 'CRR143992_r2.fastq.gz']


# Process fastq files (modify kb command according to fastqs list)


In [ ]:
fastqs

['CRR143977_f1.fastq.gz',
 'CRR143977_r2.fastq.gz',
 'CRR143978_f1.fastq.gz',
 'CRR143978_r2.fastq.gz',
 'CRR143979_f1.fastq.gz',
 'CRR143979_r2.fastq.gz',
 'CRR143980_f1.fastq.gz',
 'CRR143980_r2.fastq.gz',
 'CRR143981_f1.fastq.gz',
 'CRR143981_r2.fastq.gz',
 'CRR143982_f1.fastq.gz',
 'CRR143982_r2.fastq.gz',
 'CRR143983_f1.fastq.gz',
 'CRR143983_r2.fastq.gz',
 'CRR143984_f1.fastq.gz',
 'CRR143984_r2.fastq.gz',
 'CRR143985_f1.fastq.gz',
 'CRR143985_r2.fastq.gz',
 'CRR143986_f1.fastq.gz',
 'CRR143986_r2.fastq.gz',
 'CRR143987_f1.fastq.gz',
 'CRR143987_r2.fastq.gz',
 'CRR143988_f1.fastq.gz',
 'CRR143988_r2.fastq.gz',
 'CRR143989_f1.fastq.gz',
 'CRR143989_r2.fastq.gz',
 'CRR143990_f1.fastq.gz',
 'CRR143990_r2.fastq.gz',
 'CRR143991_f1.fastq.gz',
 'CRR143991_r2.fastq.gz',
 'CRR143992_f1.fastq.gz',
 'CRR143992_r2.fastq.gz']

In [ ]:
if no_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
  + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(no_samples):

    fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
    # Write the kb count command as a string
    cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
    --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

    # Execute it
    !$cmd

    # Update j to move to the next set of fastq
    j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd


import time
time.sleep(60000000)

[2021-04-28 19:27:42,636]    INFO Using index index.idx to generate BUS file to tccoutputCRX118014 from
[2021-04-28 19:27:42,636]    INFO         CRR143977_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143977_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143978_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143978_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143979_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143979_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143980_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143980_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143981_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143981_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143982_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143982_r2.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143983_f1.fastq.gz
[2021-04-28 19:27:42,637]    INFO         CRR143983_r2.fastq.gz


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


  adding: bus_raw_CRX118014_from_gc/ (stored 0%)
  adding: bus_raw_CRX118014_from_gc/kb_info.json (deflated 80%)
  adding: bus_raw_CRX118014_from_gc/run_info.json (deflated 68%)
  adding: bus_raw_CRX118014_from_gc/CRX118014.unfiltered.bus.gz (deflated 3%)
  adding: bus_raw_CRX118014_from_gc/tcc_unfiltered/ (stored 0%)
  adding: bus_raw_CRX118014_from_gc/tcc_unfiltered/cells_x_tcc.mtx.gz (deflated 0%)
  adding: bus_raw_CRX118014_from_gc/tcc_unfiltered/adata.h5ad.gz (deflated 5%)
  adding: bus_raw_CRX118014_from_gc/tcc_unfiltered/cells_x_tcc.ec.txt.gz (deflated 2%)
  adding: bus_raw_CRX118014_from_gc/tcc_unfiltered/cells_x_tcc.barcodes.txt.gz (deflated 1%)
  adding: bus_raw_CRX118014_from_gc/inspect.json (deflated 57%)
